In [ ]:
!python --version

Python 3.10.12


In [ ]:
!pip install datasets transformers[sentencepiece] sacrebleu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 41.1 MB/s eta 0:00:00


In [ ]:
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

In [ ]:
model_uri = "Helsinki-NLP/opus-mt-en-hi"
dataset = "cfilt/iitb-english-hindi"

In [ ]:
raw_data = load_dataset(dataset)

Generating train split:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/520 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [ ]:
print(raw_data['train'][0])

{'translation': {'en': 'Give your application an accessibility workout', 'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'}}


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_uri)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
print(tokenizer(['Give your application an accessibility workout']))

{'input_ids': [[3872, 85, 2501, 132, 15441, 36398, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1]]}


In [ ]:
print(tokenizer(text_target = ['अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें']))

{'input_ids': [[63, 2025, 18, 16155, 346, 20311, 24, 2279, 679, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [ ]:
max_input_len = 128
max_target_len = 128

def preprocess(data):
    inputs = [x['en'] for x in data["translation"]]
    targets = [x['hi'] for x in data["translation"]]

    model_inputs = tokenizer(inputs, max_length=max_input_len, truncation=True)

    labels = tokenizer(text_target = targets, max_length=max_target_len, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


print(preprocess(raw_data['train'][:5]))

{'input_ids': [[3872, 85, 2501, 132, 15441, 36398, 0], [32643, 28541, 36253, 0], [81, 1724, 1907, 6461, 34, 4, 5571, 10061, 0], [81, 1724, 1907, 6461, 34, 4, 3346, 10061, 0], [238, 583, 8, 4442, 26, 54, 6804, 97, 1724, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[63, 2025, 18, 16155, 346, 20311, 24, 2279, 679, 0], [26618, 16155, 346, 33383, 0], [18464, 12422, 6, 39, 22433, 10076, 69, 4553, 7788, 0], [6704, 12422, 6, 39, 22433, 10076, 69, 4553, 7788, 0], [141, 10076, 69, 38232, 15, 342, 1058, 22433, 246, 12, 2709, 78, 115, 5, 0]]}


In [ ]:
tokenized_data = raw_data.map(preprocess, batched=True)

Map:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Map:   0%|          | 0/520 [00:00<?, ? examples/s]

Map:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [ ]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_uri)

tf_model.h5:   0%|          | 0.00/306M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-hi.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [ ]:
generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=128)

In [ ]:
batch_size = 16
learning_rate = 2e-5
weight_decay = 0.01
num_epochs = 1

In [ ]:
train_data = model.prepare_tf_dataset(
    tokenized_data["test"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)

In [ ]:
valid_data = model.prepare_tf_dataset(
    tokenized_data["validation"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)

In [ ]:
generation_data = model.prepare_tf_dataset(
    tokenized_data["validation"],
    batch_size=8,
    shuffle=False,
    collate_fn=generation_data_collator,
)

In [ ]:
adam_optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=adam_optimizer)

In [ ]:
model.fit(train_data, validation_data=valid_data, epochs=1)

Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
156/156 [==============================] - 150s 405ms/step - loss: 3.7589 - val_loss: 3.9502


In [ ]:
model.save_pretrained("ENG_HIN_MODEL/")
tokenizer.save_pretrained("ENG_HIN_TOKENIZER/")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61949]], 'forced_eos_token_id': 0}


('ENG_HIN_TOKENIZER/tokenizer_config.json',
 'ENG_HIN_TOKENIZER/special_tokens_map.json',
 'ENG_HIN_TOKENIZER/vocab.json',
 'ENG_HIN_TOKENIZER/source.spm',
 'ENG_HIN_TOKENIZER/target.spm',
 'ENG_HIN_TOKENIZER/added_tokens.json')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("ENG_HIN_TOKENIZER/")
model = TFAutoModelForSeq2SeqLM.from_pretrained("ENG_HIN_MODEL/")

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at ENG_HIN_MODEL/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [ ]:
article = '''
The lion is a wild terrestrial animal called the king of the forest.
The lion is a strong animal with a strong body, a big head, a majestic mane, and two fierce eyes.
Lions are predatory animals and eat only after hunting.
They have strong claws and sharp teeth, which help them hunt their prey and eat the flesh.
'''

tokenized = tokenizer([article], return_tensors='np')
out = model.generate(**tokenized, max_length=128)
print(out)

tf.Tensor(
[[61949  7854  3997    24   473 10142     5    40  7854    38 11806  4597
      2    38 11806  1944     2  7965     9   214  9995  8281   310   200
      5    40  7854 18327  4597  3537     6   647    58  2023    28    43
    128   173  3186  3842     9  5831 14467    28     2    41   159 24291
    472  5171  1759     9  5171  1759    11  1532   119    28    40     0]], shape=(1, 60), dtype=int32)


In [ ]:
print(tokenizer.decode(out[0], skip_special_tokens=True))

सिंह जंगल का राजा कहलाता है। सिंह एक बलवन्त पशु, एक बलवन्त सिर, सुन्दर और दो मोटी आंखें लिये हुए है। सिंह शिकारी पशु शिकार के पीछे ही खाते हैं; उनके पास कठोर कानून और तेज दांत हैं, जो उन्हें अहेर करके मांस खाने और मांस खाने में सहायता करते हैं।
